In [32]:
import pandas as pd
import numpy as np

In [33]:
df = pd.read_csv('data/owid-covid-data.csv')

**i) Nhóm câu hỏi liên quan đến tổng quát dữ liệu**

In [34]:
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

**(1) Số lượng mẫu thu thập vào các năm nào?**


In [35]:
years = df['date'].dt.year.dropna().astype(int).unique()
print(list(years))

[2020, 2021, 2022]


**(2) Số lượng đất nước và định danh của mỗi đất nước (hiển thị 10 đất nước đầu tiên)**

Do trong file dữ liệu có lẫn các **iso_code** của châu lục, của thế giới và của nhóm các quốc gia nên để đếm số quốc gia, trước hết ta cần tìm **iso_code** có độ dài lớn hơn **3**.

In [36]:
filtered_iso_codes = df[df['iso_code'].str.len() > 3]['iso_code'].unique()

iso_codes_list = pd.DataFrame({
    'iso_code': filtered_iso_codes
})
iso_codes_list

,iso_code
0,OWID_AFR
1,OWID_ASI
2,OWID_EUR
3,OWID_EUN
4,OWID_HIC
5,OWID_INT
6,OWID_KOS
7,OWID_LIC
8,OWID_LMC
9,OWID_NAM


Ta thấy rằng trong **15 iso_code** có độ dài lớn hơn **3** thì có **2 iso_code** chỉ quốc gia là **OWID_CYN (Bắc Cyprus)** và **OWID_KOS (Kosovo)** nên để đếm số quốc gia trong dữ liệu ta đếm số giá trị phân biệt của **iso_code** rồi trừ đi **13**. Để trích ra định danh của 10 nước đầu tiên, ta ghép 2 cột **location** và **iso_code** lại tạo thành một dataframe và trích ra 10 cột đầu tiên của nó.

In [37]:
filtered_iso_codes = df[df['continent'].notna()]['iso_code'].unique()
filtered_locations = df[df['continent'].notna()]['location'].unique()

iso_code = list(filtered_iso_codes[:10]) + ["Count"]
country = list(filtered_locations[:10]) + [len(filtered_iso_codes) - 13]

country_list = pd.DataFrame({
    'iso_code': iso_code,
    'Country': country
})

country_list


,iso_code,Country
0,AFG,Afghanistan
1,ALB,Albania
2,DZA,Algeria
3,AND,Andorra
4,AGO,Angola
5,AIA,Anguilla
6,ATG,Antigua and Barbuda
7,ARG,Argentina
8,ARM,Armenia
9,ABW,Aruba


**(3) Số lượng châu lục (continent) trong tập mẫu**

In [38]:
continents = df['continent'].dropna().unique()

def continent_name(c):
    match c:
        case "Asia":
            return "Châu Á"
        case "Europe":
            return "Châu Âu"
        case "Africa":
            return "Châu Phi"
        case "Oceania":
            return "Châu Đại Dương"
        case "North America":
            return "Bắc Mỹ"
        case "South America":
            return "Nam Mỹ"
        case _:
            return c 

continent_names = pd.DataFrame(continents, columns=['continent'])
continent_names['Name'] = continent_names['continent'].apply(continent_name)

total_row = pd.DataFrame({'continent': ['Total'], 'Name': [len(continents)]})
continent_names = pd.concat([continent_names, total_row], ignore_index=True)

continent_names

,continent,Name
0,Asia,Châu Á
1,Europe,Châu Âu
2,Africa,Châu Phi
3,North America,Bắc Mỹ
4,South America,Nam Mỹ
5,Oceania,Châu Đại Dương
6,Total,6


**(4) Số lượng dữ liệu thể hiện thu thập dữ liệu được trong từng châu lục và tổng số**

In [39]:
observations = [len(df[df['continent'] == continent]) for continent in continents]
c = continents.tolist() + ["Total"]
o = observations + [len(df)]

continent_observations = pd.DataFrame({
    'Continent': c,
    'Observations': o
})

continent_observations

,Continent,Observations
0,Asia,35528
1,Europe,36375
2,Africa,38647
3,North America,24438
4,South America,9335
5,Oceania,8993
6,Total,163090


**(5) Số lượng dữ liệu thể hiện thu thập dữ liệu được trong từng từng đất nước (hiển thị 10 dất nước cuối cùng) và tổng số ca mắc của tất cả quốc gia**

Do trong file dữ liệu có chứa các **iso_code** không chỉ quốc gia **đã nêu trong câu (2)** nên ta phải trừ đi số dữ liệu thu thập được từ các **iso_code** này

In [40]:
def len_data_is_not_country(iso_codes_list):
    valid_iso_codes = iso_codes_list[~iso_codes_list['iso_code'].isin(['OWID_CYN', 'OWID_KOS'])]

    total_new_cases = len(df[df['iso_code'].isin(valid_iso_codes['iso_code'])]['new_cases'])

    return total_new_cases

total_new_cases = len_data_is_not_country(iso_codes_list)
total_new_cases

9774

In [41]:
filtered_location = df[df['continent'].notna()]['location'].unique()

country = list(filtered_location[-10:]) + ["Count"]
    
observations = []
    
for i in range(10):
    filtered_country_data = df[df['location'] == filtered_location[-(10 - i)]]
    observations.append(len(filtered_country_data))
    
observations.append(len(df) - total_new_cases)
    
country_observation = pd.DataFrame({
    'Country': country,
    'Observations': observations
})
    
country_observation

,Country,Observations
0,Uruguay,709
1,Uzbekistan,707
2,Vanuatu,467
3,Vatican,716
4,Venezuela,708
5,Vietnam,759
6,Wallis and Futuna,489
7,Yemen,681
8,Zambia,704
9,Zimbabwe,702


**(6) Cho biết các châu lục nào có lượng dữ liệu thu thập nhỏ nhất và giá trị nhó nhất đó?**

Ta có thể sử dụng lại dữ liệu đã lọc từ câu (4) và tìm số lượng dữ liệu nhỏ nhất.

In [43]:
min_observation = continent_observations['Observations'].iloc[:-1].min()
min_observation

8993

In [ ]:
min_observation_index = continent_observations['Observations'].iloc[:-1].idxmin()
continent_with_min_data = continent_observations.loc[min_observation_index, 'Continent']
continent_with_min_data

'Oceania'

**(7) Cho biết các châu lục nào có lượng dữ liệu thu thập lớn nhất và giá trị lớn nhất đó?**

In [44]:
max_observation = continent_observations['Observations'].iloc[:-1].max()
max_observation

38647

In [ ]:
max_observation_index = continent_observations['Observations'].iloc[:-1].idxmax()
continent_with_max_data = continent_observations.loc[max_observation_index, 'Continent']
continent_with_max_data

'Africa'

**(8) Cho biết các nước nào có lượng dữ liệu thu thập nhỏ nhất và giá trị nhó nhất đó?**

In [ ]:

countries = df['location'].unique()

observations = []
    
for country in countries:
    country_data = df[df['location'] == country]
    observations.append(len(country_data))
    
observations

min_observation = min(observations)
country_with_min_observation = [country for country, obs in zip(countries, observations) if obs == min_observation]

min = pd.DataFrame({
    'Country': country_with_min_observation,
    'Min Observations': min_observation
})
min

,Country,Min Observations
0,Pitcairn,85


In [ ]:
max_observation = max(observations)
country_with_max_observation = [country for country, obs in zip(countries, observations) if obs == max_observation]

max = pd.DataFrame({
    'Country': country_with_max_observation,
    'Max Observations': max_observation
})
max

,Country,Max Observations
0,Argentina,781
1,Mexico,781


**(10) Cho biết các date nào có lượng dữ liệu thu thập nhỏ nhất và giá trị nhỏ nhất đó?**

In [ ]:

date_counts = df["date"].value_counts()
min_count = date_counts.min()
result_df = date_counts[date_counts == min_count].reset_index()

result_df


,date,count
0,2020-01-03,2
1,2020-01-01,2
2,2020-01-02,2


**(11) Cho biết các date nào có lượng dữ liệu thu thập lớn nhất và giá trị lớn nhất đó?**

In [ ]:
date_counts = df["date"].value_counts()
max_count = date_counts.max()
result_df = date_counts[date_counts == max_count].reset_index()

result_df

,date,count
0,2021-08-27,238
1,2021-08-26,238
2,2021-08-22,238
3,2021-08-23,238
4,2021-08-24,238
5,2021-08-29,238
6,2021-08-28,238
7,2021-08-25,238


**(12) Cho biết số lượng dữ liệu thu thập được theo date và châu lục**

In [ ]:
continent_date_counts = df.groupby(["date", "continent"]).size().reset_index(name="observation")

continent_date_counts

,date,continent,observation
0,2020-01-01,North America,1
1,2020-01-01,South America,1
2,2020-01-02,North America,1
3,2020-01-02,South America,1
4,2020-01-03,North America,1
...,...,...,...
4595,2022-02-19,Asia,48
4596,2022-02-19,Europe,49
4597,2022-02-19,North America,34
4598,2022-02-19,Oceania,16


**(13) Cho biết số lượng dữ liệu thu thập được là lớn nhất theo date và châu lục**

In [55]:
max_date_continent = continent_date_counts.max().loc["observation"]
max_date_continent

55

**(14) Cho biết số lượng dữ liệu thu thập được là nhỏ nhất theo date và châu lục**

In [56]:
min_date_continent = continent_date_counts.min().loc["observation"]
min_date_continent

1

**(15) Với một date là k và một châu lục t cho trước, hãy cho biết số lượng dữ liệu thể hiện thu thập dữ liệu được**

In [62]:
k = "2022-02-19"
t = "Oceania"

In [63]:
result = continent_date_counts.loc[(continent_date_counts["date"] == k) & (continent_date_counts["continent"] == t)]
result 

,date,continent,observation
4598,2022-02-19,Oceania,16


**(16) Có đất nước nào mà số lượng dữ liệu thu thập được là bằng nhau không? Hãy cho biết các iso_code của đất nước đó**

In [65]:
country_counts = df["iso_code"].value_counts()

equal_counts = country_counts[country_counts.duplicated(keep=False)]

result_df = equal_counts.reset_index()

result_df


,iso_code,count
0,ARG,781
1,MEX,781
2,USA,760
3,KOR,760
4,OWID_HIC,760
...,...,...
188,AIA,694
189,BDI,691
190,SLE,691
191,SPM,686


**(17) Liệt kê iso_code, tên đất nước mà chiều dài iso_code lớn hơn 3**

In [74]:
long_iso_codes = df[df["iso_code"].str.len() > 3][["iso_code", "location"]].drop_duplicates()

long_iso_codes


,iso_code,location
727,OWID_AFR,Africa
7980,OWID_ASI,Asia
48372,OWID_EUR,Europe
49131,OWID_EUN,European Union
64660,OWID_HIC,High income
69804,OWID_INT,International
79143,OWID_KOS,Kosovo
86946,OWID_LIC,Low income
87674,OWID_LMC,Lower middle income
109152,OWID_NAM,North America
